In [ ]:
# needed only on kaggle.com
!pip install llvmlite --ignore-installed --quiet

In [ ]:
!pip install git+https://github.com/privML/privacy-evaluator.git@feat/119-slicing-advantage-score --quiet

In [ ]:
import numpy as np
import torch

import privacy_evaluator.models.torch.dcti.dcti as torch_dcti 
from privacy_evaluator.datasets.cifar10 import CIFAR10
from privacy_evaluator.classifiers.classifier import Classifier

In [ ]:
# CIFAR datasets
x_train, y_train, x_test, y_test = CIFAR10.numpy(model_type='torch')

# Wrapped Pytorch target model
target_model = Classifier(
    torch_dcti.load_dcti(), 
    loss=torch.nn.CrossEntropyLoss(), 
    nb_classes=CIFAR10.N_CLASSES, 
    input_shape=CIFAR10.INPUT_SHAPE)

In [ ]:
from privacy_evaluator.attacks.membership_inference import MembershipInferenceAttackAnalysis
from privacy_evaluator.attacks.membership_inference import MembershipInferenceBlackBoxRuleBasedAttack

from privacy_evaluator.attacks.membership_inference.data_structures.attack_input_data import AttackInputData
from privacy_evaluator.attacks.membership_inference.data_structures.slicing import Slicing

In [ ]:
input_data = AttackInputData(x_train, y_train, x_test, y_test)
attack_analysis = MembershipInferenceAttackAnalysis(MembershipInferenceBlackBoxRuleBasedAttack, input_data)

x = np.concatenate((x_train, x_test))
y = np.concatenate((y_train, y_test))
membership = np.concatenate((np.ones(len(x_train)), np.zeros(len(x_test))))
slicing = Slicing(entire_dataset=True, by_class=True, by_classification_correctness=True)

result = attack_analysis.analyse(target_model, x, y, membership, slicing)

In [ ]:
print("\n".join((str(r) for r in result)))